In [1]:
import re
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from sgp4.api import Satrec
from sgp4.ext import invjday
from astropy.time import Time
from astropy import units as u
from sgp4.conveniences import jday
from astropy.coordinates import ITRS
from datetime import datetime, timedelta
from astropy.coordinates import TEME, CartesianDifferential, CartesianRepresentation

np.set_printoptions(precision=2)
username = 'asas4539@hanyang.ac.kr'
password = 'onsaemiro1729!!'

In [105]:
def get_tle(sat_num):
    '''
    :param sat_num: 위성번호
    :return: KVN과 TLE 데이터
    '''
    # Space-Track API의 URL
    url = "https://www.space-track.org/ajaxauth/login"

    # 요청에 필요한 인증 정보
    payload = {"identity": username, "password": password}

    # Session 객체 생성
    session = requests.Session()

    # 로그인 요청
    response = session.post(url, data=payload)
    if response.status_code != 200:
        print("로그인에 실패했습니다.")
        return None

    # KVN 요청
    kvn_response = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{sat_num}/orderby/EPOCH%20ASC/EPOCH/2022-12-31--2024-08-01/format/kvn"
    )
    # TLE 요청
    tle_response = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{sat_num}/orderby/EPOCH%20ASC/EPOCH/2022-12-31--2024-08-01/format/3le"
    )

    return kvn_response, tle_response

In [158]:
def compute_change_df(kvn_response, tle_response, real_time: bool):
    tle_datas = tle_response.text.split('\r\n')
    creation_dates = list(re.findall(r"CREATION_DATE\s*=\s*([\d\-T:]+)", kvn_response.text))

    info_df = pd.DataFrame([creation_dates, tle_datas[1:][::3], tle_datas[2:][::3]]).T
    info_df.columns = ['creation_date', 'first_line', 'second_line']
    info_df['created_year'] = info_df['creation_date'].map(lambda x: int(x.split('-')[0]))
    info_df['created_month'] = info_df['creation_date'].map(lambda x: int(x.split('-')[1]))
    info_df['created_day'] = info_df['creation_date'].map(lambda x: int(x.split('-')[2].split('T')[0]))
    info_df['created_hour'] = info_df['creation_date'].map(lambda x: int(x.split('T')[1].split(':')[0]))
    info_df['created_minute'] = info_df['creation_date'].map(lambda x: int(x.split(':')[1]))
    info_df['created_second'] = info_df['creation_date'].map(lambda x: int(x.split(':')[2]))
    info_df = info_df.sort_values(by='creation_date').reset_index(drop=True)

    change_times = []
    for i in range(len(info_df)):
        tle_1, tle_2 = info_df['first_line'].iloc[i], info_df['first_line'].iloc[i]
        satellite = Satrec.twoline2rv(tle_1, tle_2)
        jdsatepoch = satellite.jdsatepoch
        jdsatepochfrac = satellite.jdsatepochF
        epochdatetime = invjday(jdsatepoch + jdsatepochfrac)
        change_times.append(epochdatetime)

    change_df = pd.DataFrame(change_times)
    change_df.columns = ['year', 'month', 'day', 'hour', 'minute', 'second']
    change_df.insert(0, 'first_line', info_df['first_line'])
    change_df.insert(1, 'second_line', info_df['second_line'])
    change_df['epoch_date'] = pd.to_datetime(change_df[['year', 'month', 'day', 'hour', 'minute', 'second']])

    if real_time:
        change_df = pd.concat([change_df, info_df[
            ['created_year', 'created_month', 'created_day', 'created_hour', 'created_minute',
             'created_second', 'creation_date']]], axis=1)
        change_df = change_df.sort_values(by='creation_date').reset_index(drop=True)
        change_df = change_df.loc[change_df[['year', 'month', 'day', 'hour', 'minute', 'second']].drop_duplicates(keep='first').index].copy()

    end_point = change_df.iloc[-1:].copy()
    end_point['month'] = 8
    end_point['day'] = 1
    end_point['hour'] = 0
    end_point['minute'] = 0
    end_point['second'] = 0
    end_point['epoch_date'] = pd.to_datetime(end_point[['year', 'month', 'day', 'hour', 'minute', 'second']])
    change_df = pd.concat([change_df, end_point], axis=0).copy()

    if not real_time:
        change_df = change_df.sort_values(by='epoch_date').reset_index(drop=True)
        change_df = change_df.loc[change_df[['year', 'month', 'day', 'hour', 'minute', 'second']].drop_duplicates(keep='first').index].copy()

    # change_df[['creation_date', 'created_year', 'created_month', 'created_day', 'created_hour', 'created_minute',
    #            'created_second']] = (
    #     info_df)[['creation_date', 'created_year', 'created_month', 'created_day', 'created_hour', 'created_minute',
    #               'created_second']]

    # change_df = change_df.sort_values(by='creation_date').reset_index(drop=True)

    # change_df = change_df[
    #     ['first_line', 'second_line', 'time', 'year', 'month', 'day', 'hour', 'minute', 'second', 'creation_date',
    #      'created_year', 'created_month', 'created_day', 'created_hour', 'created_minute', 'created_second']]
    # day_change_idx = list(change_df[['year', 'month', 'day']].drop_duplicates().index)
    return change_df

In [159]:
number = 58400
kvn_response, tle_response = get_tle(number)
change_df = compute_change_df(kvn_response, tle_response, True)

In [160]:
change_df

,first_line,second_line,year,month,day,hour,minute,second,epoch_date,created_year,created_month,created_day,created_hour,created_minute,created_second,creation_date
0,1 58400U 23179A 23326.01002942 .00008068 0...,2 58400 97.4276 210.7863 0014260 279.3296 80...,2023,11,22,0,14,26.541870,2023-11-22 00:14:26.541869939,2023,11,22,3,26,15,2023-11-22T03:26:15
1,1 58400U 23179A 23326.16510777 .00018821 0...,2 58400 97.4263 210.9434 0014062 277.7388 210...,2023,11,22,3,57,45.311319,2023-11-22 03:57:45.311318636,2023,11,22,4,6,16,2023-11-22T04:06:16
2,1 58400U 23179A 23326.95239345 .00014863 0...,2 58400 97.4261 211.7175 0014282 275.5273 201...,2023,11,22,22,51,26.794066,2023-11-22 22:51:26.794066429,2023,11,23,0,56,19,2023-11-23T00:56:19
3,1 58400U 23179A 23326.86521155 .00009675 0...,2 58400 97.4275 211.6299 0014281 275.8375 84...,2023,11,22,20,45,54.277933,2023-11-22 20:45:54.277932644,2023,11,23,3,43,51,2023-11-23T03:43:51
4,1 58400U 23179A 23327.21599119 .00012574 0...,2 58400 97.4263 211.9766 0014186 274.4179 205...,2023,11,23,5,11,1.638831,2023-11-23 05:11:01.638830602,2023,11,23,5,56,15,2023-11-23T05:56:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,1 58400U 23179A 24213.18365506 .00008690 0...,2 58400 97.3982 99.2875 0003254 70.4061 289...,2024,7,31,4,24,27.797175,2024-07-31 04:24:27.797175050,2024,7,31,6,56,18,2024-07-31T06:56:18
1155,1 58400U 23179A 24213.57820414 .00007822 0...,2 58400 97.3981 99.6755 0003225 69.4820 290...,2024,7,31,13,52,36.837702,2024-07-31 13:52:36.837701798,2024,7,31,15,6,16,2024-07-31T15:06:16
1156,1 58400U 23179A 24213.84123612 .00009173 0...,2 58400 97.3981 99.9341 0003211 69.0697 291...,2024,7,31,20,11,22.800776,2024-07-31 20:11:22.800775767,2024,7,31,23,6,17,2024-07-31T23:06:17
1157,1 58400U 23179A 24213.97275171 .00009594 0...,2 58400 97.3981 100.0633 0003201 68.8851 291...,2024,7,31,23,20,45.747752,2024-07-31 23:20:45.747752488,2024,8,1,5,40,1,2024-08-01T05:40:01


In [163]:
def compute_result_df(change_df):
    col_name = ['time',
                'x', 'y', 'z',
                'vx', 'vy', 'vz',
                'altitude', 'velocity',
                'x_earth', 'y_earth', 'z_earth',
                'vx_earth', 'vy_earth', 'vz_earth',
                'apogee', 'perigee',
                'inclination', 'eccentricity', 'raan',
                'lon', 'lat', 'height']

    all_info = []

    for i in tqdm(range(len(change_df) - 1)):

        sat_info = change_df.iloc[i]
        next_sat_info = change_df.iloc[i + 1]

        tle_1, tle_2 = sat_info['first_line'], sat_info['second_line']

        satellite = Satrec.twoline2rv(tle_1, tle_2)
        jdsatepoch = satellite.jdsatepoch
        jdsatepochfrac = satellite.jdsatepochF
        epochdatetime = list(invjday(jdsatepoch + jdsatepochfrac))
        epochdatetime[-1] = int(epochdatetime[-1])

        next_tle_1, next_tle_2 = next_sat_info['first_line'], next_sat_info['second_line']

        next_satellite = Satrec.twoline2rv(next_tle_1, next_tle_2)
        next_jdsatepoch = next_satellite.jdsatepoch
        next_jdsatepochfrac = next_satellite.jdsatepochF
        next_epochdatetime = list(invjday(next_jdsatepoch + next_jdsatepochfrac))
        next_epochdatetime[-1] = int(next_epochdatetime[-1])

        earth_radius = 6378.137
        rad2deg = 180.0 / 3.141592653589793

        start = datetime(*epochdatetime)

        if i == len(change_df) - 2:
            next_epochdatetime = [2024, 8, 1, 0, 0, 0]
            end = datetime(*next_epochdatetime)
        else:
            end = datetime(*next_epochdatetime)
            end += timedelta(seconds=1)

        start += timedelta(minutes=1)
        start -= timedelta(seconds=start.second)

        epoch = start

        jd_lst = []
        fr_lst = []

        time_lst = []

        while (epoch < end):
            year = epoch.year
            month = epoch.month
            date = epoch.day
            hour = epoch.hour
            minute = epoch.minute
            second = epoch.second

            time_lst.append(epoch)

            jd, fr = jday(year, month, date, hour, minute, second)

            jd_lst.append(jd)
            fr_lst.append(fr)

            epoch += timedelta(minutes=1)

        e, r, v = satellite.sgp4_array(np.array(jd_lst), np.array(fr_lst))

        t_lst = Time(list(np.array(jd_lst) + np.array(fr_lst)), format='jd')

        teme_p = CartesianRepresentation(r[:, 0] * u.km, r[:, 1] * u.km, r[:, 2] * u.km)
        teme_v = CartesianDifferential(v[:, 0] * u.km / u.s, v[:, 1] * u.km / u.s, v[:, 2] * u.km / u.s)
        teme = TEME(teme_p.with_differentials(teme_v), obstime=t_lst)
        print(t_lst)
        itrs_geo = teme.transform_to(ITRS(obstime=t_lst))
        locations = itrs_geo.earth_location
        geodetic_coords = locations.geodetic

        all_info.append(pd.DataFrame([time_lst,
                                      r[:, 0], r[:, 1], r[:, 2],
                                      v[:, 0], v[:, 1], v[:, 2],
                                      (r[:, 0] ** 2 + r[:, 1] ** 2 + r[:, 2] ** 2) ** 0.5,
                                      (v[:, 0] ** 2 + v[:, 1] ** 2 + v[:, 2] ** 2) ** 0.5,
                                      teme_p.x.value, teme_p.y.value, teme_p.z.value,
                                      teme_v.d_x.value, teme_v.d_y.value, teme_v.d_z.value,
                                      [satellite.alta * earth_radius] * len(time_lst),
                                      [satellite.altp * earth_radius] * len(time_lst),
                                      [satellite.inclo * rad2deg] * len(time_lst), [satellite.ecco] * len(time_lst),
                                      [satellite.nodeo] * len(time_lst),
                                      geodetic_coords.lon.value, geodetic_coords.lat.value,
                                      geodetic_coords.height.value,
                                      ]))

    result = pd.concat(all_info, axis=1).T

    result.columns = col_name
    result.time = pd.to_datetime(result.time)
    result = result.set_index(['time'])
    return result

In [164]:
result=compute_result_df(change_df)

  0%|          | 2/1129 [00:00<00:49, 22.65it/s]


[2460270.51 2460270.51 2460270.51 2460270.51 2460270.51 2460270.51
 2460270.51 2460270.52 2460270.52 2460270.52 2460270.52 2460270.52
 2460270.52 2460270.52 2460270.52 2460270.52 2460270.52 2460270.52
 2460270.52 2460270.52 2460270.52 2460270.52 2460270.53 2460270.53
 2460270.53 2460270.53 2460270.53 2460270.53 2460270.53 2460270.53
 2460270.53 2460270.53 2460270.53 2460270.53 2460270.53 2460270.53
 2460270.54 2460270.54 2460270.54 2460270.54 2460270.54 2460270.54
 2460270.54 2460270.54 2460270.54 2460270.54 2460270.54 2460270.54
 2460270.54 2460270.54 2460270.55 2460270.55 2460270.55 2460270.55
 2460270.55 2460270.55 2460270.55 2460270.55 2460270.55 2460270.55
 2460270.55 2460270.55 2460270.55 2460270.55 2460270.55 2460270.56
 2460270.56 2460270.56 2460270.56 2460270.56 2460270.56 2460270.56
 2460270.56 2460270.56 2460270.56 2460270.56 2460270.56 2460270.56
 2460270.56 2460270.57 2460270.57 2460270.57 2460270.57 2460270.57
 2460270.57 2460270.57 2460270.57 2460270.57 2460270.57 246027

ValueError: not enough values to unpack (expected 3, got 0)